# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
orders = pd.read_csv('Orders.zip')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [27]:
# Aggregate amount spent for unique customers
amount_sum = orders.groupby(by = 'CustomerID').sum() [['amount_spent']]

# Bins
labels = ['Customer', 'Regular Customer', 'Preferred Customer', 'VIP']
cutoffs = [0, 0.50, 0.75, 0.95, 1]

orders_frq = pd.qcut(amount_sum['amount_spent'], cutoffs, labels = labels)

amount_sum = pd.merge(amount_sum, orders_frq, left_index = True, right_index = True)
amount_sum = amount_sum.reset_index()
amount_sum.columns = ['CustomerID', 'amout_spent', 'Label']
amount_sum

,CustomerID,amout_spent,Label
0,12346,77183.60,VIP
1,12347,4310.00,Preferred Customer
2,12348,1797.24,Preferred Customer
3,12349,1757.55,Preferred Customer
4,12350,334.40,Customer
...,...,...,...
4334,18280,180.60,Customer
4335,18281,80.82,Customer
4336,18282,178.05,Customer
4337,18283,2094.88,Preferred Customer


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [35]:
country_customer = orders.groupby(by = ['CustomerID','Country']).sum()
country_customer = country_customer.reset_index()

# merge countries with labels on CustomerID
c_a_merge = country_customer.merge(amount_sum, on = ['CustomerID'])

#select the country with the most VIPs
#select only VIP, groupby Country, count labels
VIP = c_a_merge[c_a_merge['Label'] == 'VIP']
VIP_country = VIP.groupby(by = 'Country').count()['Label']

# sort the values, select index 0
VIP_country = VIP_country.sort_values(ascending=False).index [0]

VIP_country 

'United Kingdom'

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [54]:
#select preferred customers
preferred = c_a_merge[(c_a_merge['Label'] == 'Preferred Customer')]

#concat VIP and preferred customers
VIP_preferred = [VIP, preferred]
VIP_P_country = pd.concat(VIP_preferred)

# groupby country, count labels
VIP_P_country = VIP_P_country.groupby(by = 'Country').count()['Label']
 
# sort the values, select index 0
VIP_P_country = VIP_P_country.sort_values(ascending=False).index [0]
VIP_P_country

'United Kingdom'